## Well Logs - Porosity and permeability calculation

In [ ]:
# import the needed libraries
import pandas as pd
import matplotlib.pyplot as plt

### First, I calculate porosity for the P02 well and then I will build a function to calculate the same for all the wells

In [ ]:
# Loading the data from the txt file for the well P02
df = pd.read_csv('logs/P02_welllogs.txt', header = 3, sep = '\t') 
df.head(10)

### Assumptions:

1) As Saikumar said, the values they provided in logs are in "micronsec/meter" but they mentioned it ad micron/sec.

2) The lithology is shale 

In [ ]:
# rename the column title unit to microsec/meter
df = df.rename(columns={'depth(m)':'Depth [m]', ' transit time (micron/sec)':'Transit time [microsec/meter]'})
df.head(10)

In [ ]:
# build the Transit time column in microsec/fet
df["Transit time [microsec/ft]"] = df["Transit time [microsec/meter]"]/3.2808399
df.head()

In [ ]:
# built the Vp
df["Vp [ft/sec]"] = (1/df["Transit time [microsec/ft]"])*10**6
df.head()

### Calculate the Porosity based on the Wyllie formula; 
### Assumptions: Shale	∆tma	60 μsec/ft	
###                          Brine	∆tp	    180,5 μsec/ft	

![title](Formula_Wyllie.png)

In [ ]:
#Calculate the Porosity based on the Wyllie formula
df["Porosity - Wyllie"] = (df["Transit time [microsec/ft]"]-60)/(180.5-60)
df.head()

In [ ]:
# plot the porosity
df.plot(x = "Porosity - Wyllie", y = "Depth [m]", c='blue', lw=1, legend=False, figsize=(7,10))

#plt.ylim(2130, 2200)
#plt.xlim(0,0.25)

plt.title('Porosity Plot')
plt.gca().invert_yaxis() #invert the y axis to increase depth as we go down
plt.show()

### Build a function that will take the file name of the well logs as input and then it will calculate both porosity and permeability. For permeability, I have used the formula (6) from the paper : https://www.sciencedirect.com/science/article/pii/S0306261919312437#b0160

In [ ]:
# Build a function that will take the file name of the well logs as input and then it will calculate both porosity
# and permeability. For permeability, I have used the formula (6) from the paper : https://www.sciencedirect.com/science/article/pii/S0306261919312437#b0160
 
def por_perm_function (well_log_filename):
    
    df = pd.read_csv(well_log_filename, header = 3, sep = '\t')
    df = df.rename(columns={'depth(m)':'Depth [m]', ' transit time (micron/sec)':'Transit time [microsec/meter]'})
    df["Transit time [microsec/ft]"] = df["Transit time [microsec/meter]"]/3.2808399
    df["Vp [ft/sec]"] = (1/df["Transit time [microsec/ft]"])*10**6
    df["Porosity - Wyllie"] = (df["Transit time [microsec/ft]"]-60)/(180.5-60)*100
    
    # calculate permeability using the formula from the paper
    
    df["Permeability [mDa]"] = (10**(-2.03*10**(-7)*df["Porosity - Wyllie"].pow(5)+2.55*10**(-5)*df["Porosity - Wyllie"].pow(4)-1.04*10**(-3)*df["Porosity - Wyllie"].pow(3)+8.91*10**(-3)*df["Porosity - Wyllie"].pow(2)+3.58*10**(-1)*df["Porosity - Wyllie"]-3.21*df["Porosity - Wyllie"].pow(0)))
    #(10.^(-2.03*10^-7*ran.^5+2.55*10^-5*ran.^4-1.04*10^-3*ran.^3+8.91*10^-3*ran.^2+3.58*10^-1*ran-3.21*ran.^0))
    return df

In [ ]:
# apply the built function to all the well logs to calculate porosity and permeability
wellog_p01 = por_perm_function('logs/P01_welllogs.txt')
wellog_p02 = por_perm_function('logs/P02_welllogs.txt')
wellog_p03 = por_perm_function('logs/P03_welllogs.txt')
wellog_e02 = por_perm_function('logs/E02_welllogs.txt')
wellog_e03 = por_perm_function('logs/E03_welllogs.txt')
wellog_e05 = por_perm_function('logs/E05_welllogs.txt')
wellog_e06 = por_perm_function('logs/E06_welllogs.txt')
wellog_i01 = por_perm_function('logs/I01_welllogs.txt')
wellog_i03 = por_perm_function('logs/I03_welllogs.txt')


In [ ]:
# Plot the porosity for all the wells 

figure, axis = plt.subplots(3, 3,figsize=(15,30))

#p01
axis[0, 0].plot(wellog_p01["Porosity - Wyllie"], wellog_p01["Depth [m]"])
axis[0, 0].set_title("Porosity-PO1")
axis[0,0].set(xlim=(0, 25))
  
#p02
axis[0, 1].plot(wellog_p02["Porosity - Wyllie"], wellog_p02["Depth [m]"])
axis[0, 1].set_title("Porosity-PO2")
axis[0,1].set(xlim=(0, 25))
  
#p03
axis[0, 2].plot(wellog_p03["Porosity - Wyllie"], wellog_p03["Depth [m]"])
axis[0, 2].set_title("Porosity-PO3")
axis[0,2].set(xlim=(0, 25))
  
#e02
axis[1, 0].plot(wellog_e02["Porosity - Wyllie"], wellog_e02["Depth [m]"])
axis[1, 0].set_title("Porosity-EO2")
axis[1,0].set(xlim=(0, 25))
  
#e03
axis[1, 1].plot(wellog_e03["Porosity - Wyllie"], wellog_e03["Depth [m]"])
axis[1, 1].set_title("Porosity-EO3")
axis[1,1].set(xlim=(0, 25))

#e05
axis[1, 2].plot(wellog_e05["Porosity - Wyllie"], wellog_e05["Depth [m]"])
axis[1, 2].set_title("Porosity-EO5")
axis[1,2].set(xlim=(0, 25))

#e06
axis[2, 0].plot(wellog_e06["Porosity - Wyllie"], wellog_e06["Depth [m]"])
axis[2, 0].set_title("Porosity-EO6")
axis[2,0].set(xlim=(0, 25))

#I01
axis[2, 1].plot(wellog_i01["Porosity - Wyllie"], wellog_i01["Depth [m]"])
axis[2, 1].set_title("Porosity-I01")
axis[2,1].set(xlim=(0, 25))

#I03
axis[2, 2].plot(wellog_i03["Porosity - Wyllie"], wellog_i03["Depth [m]"])
axis[2, 2].set_title("Porosity-I03")
axis[2,2].set(xlim=(0, 25))

plt.show()

In [ ]:
# plot the permeability plot for each well
figure, axis = plt.subplots(3, 3,figsize=(15,30))

#p01
axis[0, 0].plot(wellog_p01["Permeability [mDa]"], wellog_p01["Depth [m]"])
axis[0, 0].set_title("Permeability-PO1 [mDa]")
axis[0,0].set(xlim=(0, 1000))
  
#p02
axis[0, 1].plot(wellog_p02["Permeability [mDa]"], wellog_p02["Depth [m]"])
axis[0, 1].set_title("Permeability-PO2 [mDa]")
axis[0,1].set(xlim=(0, 1000))
  
#p03
axis[0, 2].plot(wellog_p03["Permeability [mDa]"], wellog_p03["Depth [m]"])
axis[0, 2].set_title("Permeability-PO3 [mDa]")
axis[0,2].set(xlim=(0, 1000))
  
#e02
axis[1, 0].plot(wellog_e02["Permeability [mDa]"], wellog_e02["Depth [m]"])
axis[1, 0].set_title("Permeability-EO2 [mDa]")
axis[1,0].set(xlim=(0, 1000))
  
#e03
axis[1, 1].plot(wellog_e03["Permeability [mDa]"], wellog_e03["Depth [m]"])
axis[1, 1].set_title("Permeability-EO3 [mDa]")
axis[1,1].set(xlim=(0, 1000))

#e05
axis[1, 2].plot(wellog_e05["Permeability [mDa]"], wellog_e05["Depth [m]"])
axis[1, 2].set_title("Permeability-EO5 [mDa]")
axis[1,2].set(xlim=(0, 1000))

#e06
axis[2, 0].plot(wellog_e06["Permeability [mDa]"], wellog_e06["Depth [m]"])
axis[2, 0].set_title("Permeability-EO6 [mDa]")
axis[2,0].set(xlim=(0, 1000))

#I01
axis[2, 1].plot(wellog_i01["Permeability [mDa]"], wellog_i01["Depth [m]"])
axis[2, 1].set_title("Permeability-I01 [mDa]")
axis[2,1].set(xlim=(0, 1000))

#I03
axis[2, 2].plot(wellog_i03["Permeability [mDa]"], wellog_i03["Depth [m]"])
axis[2, 2].set_title("Permeability-I03 [mDa]")
axis[2,2].set(xlim=(0, 1000))

plt.show()